# 范式，减少过拟合
* bias和variance，high bias表示欠拟合，high variance表示过拟合（缺数据），并且要以贝叶斯误差为基准，如下图所示，贝叶斯误差为0
    <img src='../images/deeplearning/bias和variance.png' width='450px'>
* 算法调试基本原则：<img src='../images/deeplearning/算法调试原则.png' width='400px'>
* $L_2$范式，或者弗罗贝尼乌斯范数：减少high variance（过拟合），实际操作过程如下
    * 前向传播：$J(W, b) = \frac{1}{m}\sum^m_{i=1}L(\sigma(\hat{y}), y) + \frac{\lambda}{2m}\sum^L_{j=1}\|W^{[L]}\|^2_F$，其中$\|W^{[L]}\|^2_F=W^{[L]} \bullet {W^{[L]}}^T$
    * 反向传播：$\partial W^{[L]}=\frac{1}{m}\partial Z^{[L]} \bullet A^{[L-1]} + \frac{\lambda}{m}W^{[L]}$
    * 这个步骤也叫做权重衰减，$L_2$范式让$W$的更新每次更加减少了$\frac{\alpha \lambda}{m}$，也就是参数的更新由以前的$W'=W-\alpha \partial W$变为了$W'=(1-\frac{\alpha \lambda}{m})W - \alpha \partial W$，可以确保代价函数的值是随着迭代次数逐步递减的，因为每次$W$的更新无论如何都要变小
    * 加入$L_2$范式可以减少high variance（过拟合）的原因是当$\lambda$设置成很大时，$W$的影响就会趋于0，会把很多隐藏层的功能抵消，将深度网络简化成一个简单的神经网络，甚至简化变成本身的机器学习模型；当$\lambda$设置成很小时，$W$的影响就会趋于1，就是原本的深度网络模型；因此在这个调整过程中，总会有一个$\lambda$的值让模型结果既不会high bias（欠拟合），也不会high variance（过拟合）
    * $\lambda$设置成很大时，$W$的影响就会趋于0的原因可能是由于公式$W'=(1-\frac{\alpha \lambda}{m})W - \alpha \partial W$
    * 上式中的$\lambda$叫做正则化参数，$\alpha$叫做衰减速率，描述梯度下降的速率
* drop-out范式，对每层节点设计一个随机的drop概率参数，随机丢掉某些节点的结果，以此减少high variance（过拟合）
    * 前向传播：$A^{[L]}_{new} = A^{[L]} \ast D^{[L]}, A^{[L]}_{new} = \frac{A^{[L]}_{new}}{D^{[L]}}$，$D^{[L]}$和$A^{[L]}$是同一纬度的矩阵，是按照drop-out概率生成的随机0和1元素矩阵，0表示过滤，1表示保留，并且每层网络这个drop-out概率不一样；这两个矩阵相乘是元素相乘，也就是每个元素乘以它对应的那个元素，例如$a_1 \ast d_1$就是新的第一个元素的值；
    * 反向传播：$\partial A^{[L]}_{new} = \partial A^{[L]} \ast D^{[L]}, \partial A^{[L]}_{new} = \frac{\partial A^{[L]}_{new}}{D^{[L]}}$
    * 有点是用在训练数据比较少的领域，可以很好的减少high variance（过拟合），例如计算机视觉
    * 但是加上drop-out后，代价函数的值就变得不可靠了，每次的结果都会发生很大的变化，不一定是个递减函数了
* 其他范式，减少high variance（过拟合）
    * 数据集扩增(Data augmentation)：随机扭转原始图片、放大截取、加波纹等等，让原始图片稍微变型作为新的数据集
    * early-stop：就是通过观察代价函数的曲线，观察训练集和dev集的结果，出现分叉处就把迭代次数定位到分叉点，非常不好，把正交化搞混了，有在做结果调优，有在避免过拟合，这两个是分开的步骤

****

# 归一化原始数据，加速模型训练
* 归一化处理步骤：
    * 首先求出训练集的中每个变量的平均值，然后减去该平均值，公式：$\mu=\frac{1}{m}\sum^m_1 x^i, x^i=[x^i_1, x^i_2, \cdots, x^i_n]^T, \mu=[\mu_1, \mu_2, \cdots, \mu_n]^T, x^i=x^i-\mu$
    * 其次将第一步处理完的$x^i$进行方差处理，公式：$\sigma^2=\frac{1}{m}\sum^m_1 {x^i}^2, \sigma^2=[{\sigma_1}^2, {\sigma_2}^2, \cdots, {\sigma_n}^2]^T, x^i=\frac{x^i}{\sigma}$
    * 综上两步，其实是对每个训练集的每个变量做如下处理：$x_i=\frac{x_i-\mu_i}{{\sigma_i}}$，其中$\mu_i$是$x_i$的期望，${\sigma_i}$是$x_i$方差，下图演示了经过两步处理后的数据集变化：
    <table>
        <tr>
            <td><img src='..\images\deeplearning\归一化处理数据集2维.png'></td>
            <td><img src='..\images\deeplearning\归一化处理数据集3维.png'></td>
        </tr>
    </table>
    * 当归一化原始数据集时，需要保证训练集，测试集都是使用同一个$\mu$和${\sigma}^2$，所以最好求全部数据集的$\mu$和${\sigma}^2$，然后分别应用到训练集和测试集中，即使$\mu$和${\sigma}^2$是按训练集的数据求解，也要将其应用到测试集中。所以不要分别对训练集和测试集求解一对$\mu$和${\sigma}^2$
* 梯度消失和梯度爆炸：
    * 深度网络中，由于层数太多，导致代价函数的斜率可能会非常小（梯度消失），也可能非常大（梯度爆炸），随机初始化$w=\frac{1}{n}$可以部分解决这个问题，$n$是训练集中变量（特征数）的数量，更多随机初始化$w$的方法可以参考《deep-neural-network》-《01_practical-aspects-of-deep-learning》-《03_setting-up-your-optimization-problem》-《03_weight-initialization-for-deep-networks.mp4》

****

# 算法优化，加快学习速率，主要优化后向传播过程
* 小步梯度下降训练算法：将数据集才分为多份很小的数据集，选择一份数据集的大小范围在64-512之间，拆分的数据集需要能够全部装进CPU/GPU内存中：
    <img src='../images/deeplearning/小步梯度算法.png' width='450px'>
* 指数加权平均，优化梯度下降算法，官方术语叫做动量梯度下降：
    * 可以避免在纵向过度震荡，加快横向下降速度
    * 公式：$v_t = \beta v_t + (1 - \beta) \theta_t$，所求的结果相当于是$\theta_t$周围$\frac{1}{1-\beta}$个点的平均值，在小步梯度下降训练算法中，将利用该公式更新$\partial W$和$\partial B$
    * 计算过程：
        * $v_{dW} = \beta v_{dW} + (1 - \beta) \partial W, W' = W - \alpha v_{dW}$
        * $v_{db} = \beta v_{db} + (1 - \beta) \partial b, b' = b - \alpha v_{db}$
        * <b>计算动量梯度下降算法必须在小步梯度下降算法的前提下执行，也就是设置好$v_{dW}=[0], v_{db}=[0]$即可，每次迭代一个小范围样例数据，都将会更新对应的$v_{dW}, v_{db}$</b>，详情见下图：
        <img src='../images/deeplearning/动量梯度下降.png' width='350px'>
        * <b>同理针对不同层数的$W^{[L]}$，需要对应的$v_{dW}^{[L]}, v_{db}^{[L]}$，这两点对于均方根传递(RMSprop)梯度下降算法和Adam梯度下降算法同样使用</b>
* 均方根传递(RMSprop)梯度下降算法，动量梯度下降的变种：
    * $s_{dW} = \beta s_{dW} + (1 - \beta) \partial W^2, W' = W - \alpha \frac{\partial W}{\sqrt{s_{dW}} + \epsilon}$
    * $s_{db} = \beta s_{db} + (1 - \beta) \partial b^2, b' = b - \alpha \frac{\partial b}{\sqrt{s_{db}} + \epsilon}$
* Adam梯度下降算法（自适应动量梯度下降算法），将RMSprop梯度下降算法和动量梯度下降算法结合：
    * $v_{dW} = \beta_1 v_{dW} + (1 - \beta_1) \partial W, v_{db} = \beta_1 v_{db} + (1 - \beta_1) \partial b$
    * $s_{dW} = \beta_2 s_{dW} + (1 - \beta_2) \partial W^2, s_{db} = \beta_2 s_{db} + (1 - \beta_1) \partial b^2$
    * $v_{dW}^{correct} = \frac{v_{dW}}{1 - \beta_1^t}, v_{db}^{correct} = \frac{v_{db}}{1 - \beta_1^t}$
    * $s_{dW}^{correct} = \frac{s_{dW}}{1 - \beta_2^t}, s_{db}^{correct} = \frac{s_{db}}{1 - \beta_2^t}$
    * $W' = W - \alpha \frac{v_{dW}^{correct}}{\sqrt{s_{dW}^{correct}} + \epsilon}, b' = b - \alpha \frac{v_{db}^{correct}}{\sqrt{s_{db}^{correct}} + \epsilon}$
    * 详情如下图所示：参数取值推荐$\beta_1=0.9, \beta_2=0.999, \epsilon=10^{-8}, \alpha$需要自适应调整或者自己调试选择不同的值
        <img src='../images/deeplearning/adam梯度下降.png' width='500px'>
* 学习速率衰减：
    * 对学习速率$\alpha$做如下的一种运算，只需要设定$\alpha_0$，在训练后期才需要考虑，能够很好的加快学习速率：
        * $\alpha = \frac{1}{1 + \eta_{dr} * t} \alpha_0$，$\eta_{dr}$是衰减率decay_rate，$t$是小步梯度里面的迭代次数，官方术语epoch
        * $\alpha = 0.95^t * \alpha_0$，以指数级速率衰减学习速率
        * $\alpha = \frac{k}{\sqrt{t}} \alpha$，$k$是常数

****

# 调参，加快学习速率，主要优化前向传播
* 所有涉及的参数，重要程度依次减低，$\alpha$学习速率，$\beta$动量参数，$t=\frac{m}{mimi-batch}$小步梯度训练次数，$n$隐藏层神经元节点数，$L$神经网络层数，$\eta_{dr}$学习速率衰减率。一般而言关于Adam梯度下降算法中的参数$\beta_1=0.9, \beta_2=0.999, \epsilon=10^{-8}$，不需要调整
* 参数调试中，随机生成组合，然后通过区域缩小搜索法，找出相对比较优的参数组合
* 参数随机选择时，要控制好范围，尽量不要使用线性范围搜索，使用$10^n$均匀选择范围，$n$是负数，详情可以参考《deep-neural-network》-《03_hyperparameter-tuning-batch-normalization-and-programming-frameworks》-《01_hyperparameter-tuning》-《02_using-an-appropriate-scale-to-pick-hyperparameters.mp4》
* 批量归一化调参，结合小步梯度下降算法使用
    * 其核心是除了归一化输入参数外，还会针对每个隐藏层的中间结果进行归一化处理，也就是对$Z^{[L]}$进行归一化处理
    * 主要的作用是，避免协变量的影响，也就是说，当模型$X -> Y$成型后，改变输入值$X$的范围，对$Y$的预测结构会产生影响。在深度网络中，每一个隐藏层的输入参数都是来自上一层的计算结果，因此会无形增加很多协变量的影响，批量归一化可以尽可能避免这些影响，让深度网络每一层更加独立，更加健壮
    * 计算$\mu$和$\sigma$时，是用每一小步的数据集求该小步中的$\mu$和$\sigma$，因此会产生部分噪声，并且对于测试集数据，不要单独计算测试集数据的$\mu$和$\sigma$，而是直接使用训练集数据的$\mu$和$\sigma$
    * 先对$z^i$做归一化处理：<img src='../images/deeplearning/批量归一化单一步骤.png' width='200px'>
        * $\mu=\frac{1}{m}\sum^m_1 z^i, \sigma^2=\frac{1}{m}\sum^m_1 {(z^i - \mu)}^2, z_i=\frac{z_i-\mu_i}{\sqrt{{\sigma_i}^2 + \epsilon}}$，注意这里的$m$只是该小步中的训练集数量，不是整个训练集的数量
        * $\tilde{z^i}=\gamma z^i + \beta$，$\gamma$和$\beta$是不同的归一化值，是可学习的参数值，例如$\gamma=\sqrt{{\sigma_i}^2 + \epsilon}, \beta=\mu$等等，并且每次反向传播时需要更新$\gamma'=\gamma - \alpha \partial \gamma$和$\beta'=\beta - \alpha \partial \beta$
        * 详细计算示例：
        <table>
            <tr>
                <td><img src='../images/deeplearning/批量归一化0.png' width='500px'></td>
                <td><img src='../images/deeplearning/批量归一化1.png' width='500px'></td>
                <td><img src='../images/deeplearning/批量归一化2.png' width='500px'></td>
            </tr>
        </table>

****

# 多分类问题以及深度学习框架
* softmax激活函数可以用来产生不同分类结果的概率，详情参考《03_hyperparameter-tuning-batch-normalization-and-programming-frameworks》-《03_multi-class-classification》
    <table>
        <tr>
            <td><img src='../images/deeplearning/softmax激活函数0.png'></td>
            <td><img src='../images/deeplearning/softmax激活函数1.png'></td>
        </tr>
    </table>
* 深度学习框架，记住框架会帮你计算反向传播，你只需要关心前向传播的公式
    * Caffe/Caffe2, CNTK, DL4J, Keras, Lasagne, mxnet, PaddlePaddle, Tensorflow, Theano, Torch
    * Tensorflow初步了解：<img src='../images/deeplearning/tensorflow代码示例.png' width='400px'>